In [355]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.1/spark-2.4.1-bin-hadoop2.7.tgz
!tar xf spark-2.4.1-bin-hadoop2.7.tgz
!pip install -q findspark

In [356]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.1-bin-hadoop2.7"

In [357]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [358]:
from pyspark.sql import SparkSession

In [359]:
spark = SparkSession.builder \
   .appName("Neural Network Model") \
   .config("spark.executor.memory", "3gb") \
   .getOrCreate()
   
sc = spark.sparkContext

In [360]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

**2. Data Understanding using SparkSQL**

In [361]:
! wget https://storage.googleapis.com/class25jan2022/share/2008.csv

--2022-02-03 09:03:01--  https://storage.googleapis.com/class25jan2022/share/2008.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 209.85.146.128, 209.85.147.128, 142.250.136.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|209.85.146.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 689413344 (657M) [text/csv]
Saving to: ‘2008.csv.3’

2008.csv.3          100%[===================>] 657.48M   151MB/s    in 4.6s    

2022-02-03 09:03:06 (144 MB/s) - ‘2008.csv.3’ saved [689413344/689413344]



In [362]:
! wc -l ./2008.csv

7009729 ./2008.csv


In [363]:
! head -3 2008.csv

Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
2008,1,3,4,2003,1955,2211,2225,WN,335,N712SW,128,150,116,-14,8,IAD,TPA,810,4,8,0,,0,NA,NA,NA,NA,NA
2008,1,3,4,754,735,1002,1000,WN,3231,N772SW,128,145,113,2,19,IAD,TPA,810,5,10,0,,0,NA,NA,NA,NA,NA


In [364]:
raw_df = spark.read.format('csv').\
option('header','true').option('mode','DROPMALFORMED')\
.load('2008.csv')

In [365]:
raw_df.count()

7009728

In [366]:
raw_df.printSchema()

root
 |-- Year: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- DayofMonth: string (nullable = true)
 |-- DayOfWeek: string (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: string (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: string (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: string (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: string (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: string (nullable = true)
 |-- CarrierDelay:

In [367]:
raw_df\
.select(['DepTime','TaxiOut','TaxiIn',\
                     'DayOfWeek','Distance','ArrDelay'])\
                     .describe().show()

+-------+------------------+------------------+-----------------+------------------+-----------------+-----------------+
|summary|           DepTime|           TaxiOut|           TaxiIn|         DayOfWeek|         Distance|         ArrDelay|
+-------+------------------+------------------+-----------------+------------------+-----------------+-----------------+
|  count|           7009728|           7009728|          7009728|           7009728|          7009728|          7009728|
|   mean|1333.8300461105448|16.453045177492882|6.860851704974527|3.9241815088973495|726.3870294253928| 8.16845238729114|
| stddev|478.06889486629836|11.332798654232155|4.933649371300466|1.9882589459851212|562.1018034840403|38.50193694882867|
|    min|                 1|                 0|                0|                 1|              100|               -1|
|    max|                NA|                NA|               NA|                 7|              999|               NA|
+-------+------------------+----

In [368]:
raw_df.groupBy('month').count().orderBy('count',ascending=False).show()

+-----+------+
|month| count|
+-----+------+
|    7|627931|
|    3|616090|
|    8|612279|
|    6|608665|
|    5|606293|
|    1|605765|
|    4|598126|
|    2|569236|
|   10|556205|
|   12|544958|
|    9|540908|
|   11|523272|
+-----+------+



In [369]:
from pyspark.sql import functions as sparkf

In [370]:
raw_df.groupBy('month').agg({'month':'count','ArrDelay':'avg'})\
.withColumnRenamed('count(month)','numMonth')\
.withColumnRenamed('avg(ArrDelay)','avgArrDelay')\
.orderBy(sparkf.col('numMonth').desc(),sparkf.col('avgArrDelay').desc()).show()

+-----+--------+------------------+
|month|numMonth|       avgArrDelay|
+-----+--------+------------------+
|    7|  627931| 9.975049681276131|
|    3|  616090| 11.19236458018227|
|    8|  612279|  6.91091468997087|
|    6|  608665|13.266756009659792|
|    5|  606293| 5.978448290248828|
|    1|  605765|10.188855960349496|
|    4|  598126| 6.807297481094145|
|    2|  569236|13.077836997760205|
|   10|  556205|0.4154954706912698|
|   12|  544958|16.680505081496417|
|    9|  540908|0.6977328787273043|
|   11|  523272| 2.015857969430839|
+-----+--------+------------------+



In [391]:
raw_df.select(['DepTime','TaxiOut','TaxiIn',\
                     'DayOfWeek','Distance','ArrDelay']).show(10)

+-------+-------+------+---------+--------+--------+
|DepTime|TaxiOut|TaxiIn|DayOfWeek|Distance|ArrDelay|
+-------+-------+------+---------+--------+--------+
|   2003|      8|     4|        4|     810|     -14|
|    754|     10|     5|        4|     810|       2|
|    628|     17|     3|        4|     515|      14|
|    926|      7|     3|        4|     515|      -6|
|   1829|     10|     3|        4|     515|      34|
|   1940|     10|     4|        4|     688|      11|
|   1937|      7|     3|        4|    1591|      57|
|   1039|      7|     7|        4|    1591|     -18|
|    617|     19|     6|        4|     451|       2|
|   1620|      6|     3|        4|     451|     -16|
+-------+-------+------+---------+--------+--------+
only showing top 10 rows



In [372]:
raw_df2 = raw_df.withColumn("DepTime",raw_df.DepTime.cast('double'))

In [373]:
raw_df2.printSchema()

root
 |-- Year: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- DayofMonth: string (nullable = true)
 |-- DayOfWeek: string (nullable = true)
 |-- DepTime: double (nullable = true)
 |-- CRSDepTime: string (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: string (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: string (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: string (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: string (nullable = true)
 |-- CarrierDelay:

In [374]:
sparkf_converted_deptime = sparkf.udf(lambda x: 'NA' if x==None else x/100.00)

In [375]:
raw_df3 = raw_df2.withColumn('DepTime', sparkf_converted_deptime(sparkf.col('DepTime')))

In [376]:
raw_df3 = raw_df3.withColumn('DepTime1', sparkf.when((raw_df3.DepTime > 0.0) & (raw_df3.DepTime < 6.0) , '00.01-05.59').when((raw_df3.DepTime >= 6.0) & (raw_df3.DepTime < 12.0) , '06.00-11.59').when((raw_df3.DepTime >= 7.0) & (raw_df3.DepTime < 18.0) , '12.00-17.59').when((raw_df3.DepTime >= 18.0) & (raw_df3.DepTime <= 24.0) , '18.00-24.00'))

In [377]:
raw_df3.show(10)

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+-----------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|   DepTime1|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+-----------+
|2008|    1|         3|        4|  20.03|      1955|   221

In [393]:
raw_df3 = raw_df3.withColumn('DepTime1', sparkf.when((raw_df3.DepTime > 0.0) & (raw_df3.DepTime < 6.0) , '00.01-05.59').when((raw_df3.DepTime >= 6.0) & (raw_df3.DepTime < 12.0) , '06.00-11.59').when((raw_df3.DepTime >= 12.0) & (raw_df3.DepTime < 18.0) , '12.00-17.59').when((raw_df3.DepTime >= 18.0) & (raw_df3.DepTime <= 24.0) , '18.00-24.00').otherwise('null'))

In [394]:
raw_df3.groupBy('DepTime1').count().orderBy('DepTime1').show()

+-----------+-------+
|   DepTime1|  count|
+-----------+-------+
|00.01-05.59| 179949|
|06.00-11.59|2643673|
|12.00-17.59|2554672|
|18.00-24.00|1495188|
|       null| 136246|
+-----------+-------+

